In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from gensim.models import LdaModel, CoherenceModel
from gensim import corpora
import pyLDAvis.gensim

In [2]:
file = pd.read_table('~/cns/steps/discFeatures_10.txt', dtype=str)
df = file.drop(['Sample', 'Chr'], axis = 1)

In [3]:
df.head()

,CN,SegSize,Dist2Cent,SegVal,LOH,SizeDipSeg,CpCN,Dist2nCNV,GCcSeg
0,3,6,3,4,0,1,2,6,4
1,3,5,4,6,0,1,5,2,3
2,3,5,4,3,0,1,3,4,3
3,2,6,2,1,0,3,2,2,1
4,2,5,6,2,0,2,5,3,2


In [4]:
listedDf = df.values.tolist()
listedDf[0:10]

[['3', '6', '3', '4', '0', '1', '2', '6', '4'],
 ['3', '5', '4', '6', '0', '1', '5', '2', '3'],
 ['3', '5', '4', '3', '0', '1', '3', '4', '3'],
 ['2', '6', '2', '1', '0', '3', '2', '2', '1'],
 ['2', '5', '6', '2', '0', '2', '5', '3', '2'],
 ['2', '6', '4', '2', '0', '3', '1', '5', '4'],
 ['5', '5', '4', '6', '0', '1', '3', '5', '2'],
 ['1', '5', '1', '1', '1', '1', '2', '5', '4'],
 ['2', '5', '3', '1', '0', '1', '3', '6', '4'],
 ['2', '6', '1', '2', '1', '3', '1', '6', '4']]

## Using 2-15 topics

In [10]:
dirichlet_dict = corpora.Dictionary(listedDf)
bow_corpus = [dirichlet_dict.doc2bow(text) for text in listedDf]

# Considering 1-15 topics, as the last is cut off
num_topics = list(range(4)[1:])
num_keywords = 15

LDA_models = {}
LDA_topics = {}
for i in num_topics:
    LDA_models[i] = LdaModel(corpus=bow_corpus,
                             id2word=dirichlet_dict,
                             num_topics=i,
                             update_every=1,
                             chunksize=len(bow_corpus),
                             passes=20,
                             alpha='auto',
                             random_state=42)

    shown_topics = LDA_models[i].show_topics(num_topics=i, 
                                             num_words=num_keywords,
                                             formatted=False)
    LDA_topics[i] = [[word[0] for word in topic[1]] for topic in shown_topics]

In [11]:
def jaccard_similarity(topic_1, topic_2):
    """
    Derives the Jaccard similarity of two topics

    Jaccard similarity:
    - A statistic used for comparing the similarity and diversity of sample sets
    - J(A,B) = (A ∩ B)/(A ∪ B)
    - Goal is low Jaccard scores for coverage of the diverse elements
    """
    intersection = set(topic_1).intersection(set(topic_2))
    union = set(topic_1).union(set(topic_2))
                    
    return float(len(intersection))/float(len(union))

In [12]:
LDA_stability = {}
for i in range(0, len(num_topics)-1):
    jaccard_sims = []
    for t1, topic1 in enumerate(LDA_topics[num_topics[i]]): # pylint: disable=unused-variable
        sims = []
        for t2, topic2 in enumerate(LDA_topics[num_topics[i+1]]): # pylint: disable=unused-variable
            sims.append(jaccard_similarity(topic1, topic2))    
        
        jaccard_sims.append(sims)    
    
    LDA_stability[num_topics[i]] = jaccard_sims
                
mean_stabilities = [np.array(LDA_stability[i]).mean() for i in num_topics[:-1]]

In [13]:
coherences = [CoherenceModel(model=LDA_models[i], texts=listedDf, dictionary=dirichlet_dict, coherence='c_v').get_coherence()\
              for i in num_topics[:-1]]

In [14]:
coh_sta_diffs = [coherences[i] - mean_stabilities[i] for i in range(num_keywords)[:-1]] # limit topic numbers to the number of keywords
coh_sta_max = max(coh_sta_diffs)
coh_sta_max_idxs = [i for i, j in enumerate(coh_sta_diffs) if j == coh_sta_max]
ideal_topic_num_index = coh_sta_max_idxs[0] # choose less topics in case there's more than one max
ideal_topic_num = num_topics[ideal_topic_num_index]

IndexError: list index out of range

In [ ]:
plt.figure(figsize=(20,10))
ax = sns.lineplot(x=num_topics[:-1], y=mean_stabilities, label='Average Topic Overlap')
ax = sns.lineplot(x=num_topics[:-1], y=coherences, label='Topic Coherence')

ax.axvline(x=ideal_topic_num, label='Ideal Number of Topics', color='black')
ax.axvspan(xmin=ideal_topic_num - 1, xmax=ideal_topic_num + 1, alpha=0.5, facecolor='grey')

y_max = max(max(mean_stabilities), max(coherences)) + (0.10 * max(max(mean_stabilities), max(coherences)))
ax.set_ylim([0, y_max])
ax.set_xlim([1, num_topics[-1]-1])
                
ax.axes.set_title('Model Metrics per Number of Topics', fontsize=25)
ax.set_ylabel('Metric Level', fontsize=20)
ax.set_xlabel('Number of Topics', fontsize=20)
plt.legend(fontsize=20)
plt.show()   

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary) # Give the correct variables